In [ ]:
import pulp as pl
solver_list = pl.listSolvers()
print(solver_list)

In [ ]:
solver_list = pl.listSolvers(onlyAvailable=True)
print(solver_list)

In [ ]:
pyomo help --solvers

In [ ]:
import linopy
import pandas as pd
import xarray as xr
m = linopy.Model()
time = pd.Index(range(10), name='time')

x = m.add_variables(lower=0, coords=[time], name='x', )
y = m.add_variables(lower=0, coords=[time], name='y')

factor = pd.Series(time, index=time)

con1 = m.add_constraints(3*x + 7*y >= 10*factor, name='con1')
con2 = m.add_constraints(5*x + 2*y >= 3*factor, name='con2')

m.add_objective(x + 2*y)
m.solve()

m.solve();
sol = m.solution.to_dataframe()
sol.plot(grid=True, ylabel='Optimal Value')

In [ ]:
import linopy

m = linopy.Model()
import pandas as pd
time = pd.Index(range(10), name='time')

x = m.add_variables(lower=0, coords=[time], name='x', )
y = m.add_variables(lower=0, coords=[time], name='y')


In [ ]:
import linopy
import numpy as np

# 定义变量

m = linopy.Model()

x = m.add_variables(name='x',lower=0)
y = m.add_variables(name='y',lower=0)

# 定义目标函数
objective = 3 * x + 2 * y

# 定义约束
A = np.array([[1, 2], [-1, 1]])
b = np.array([2, 1])

# 添加约束到问题中

# 创建问题实例
m.add_constraints(A @ [x, y] <= b)

m.add_objective(objective)

# 求解问题
solution = m.solve()

# 打印结果
print("Solution:", solution.value)
print("Optimal values of variables:", solution.variables)


In [ ]:
from optlang.scipy_interface import Model, Variable, Constraint, Objective

# All the (symbolic) variables are declared, with a name and optionally a lower and/or upper bound.
x1 = Variable('x1', lb=0, ub = 100)
x2 = Variable('x2', lb=0)
x3 = Variable('x3', lb=0)

# A constraint is constructed from an expression of variables and a lower and/or upper bound (lb and ub).
c1 = Constraint(x1 + x2 + x3, lb = -100 ,ub=100)
c2 = Constraint(10 * x1 + 4 * x2 + 5 * x3, ub=200)
c3 = Constraint(2 * x1 + 2 * x2 + 6 * x3, ub=300)

# An objective can be formulated
obj = Objective(10 * x1 + 6 * x2 + 4 * x3, direction='max')

# Variables, constraints and objective are combined in a Model object, which can subsequently be optimized.
model = Model(name='Simple model')
model.objective = obj
# model.configuration.lp_method = 'simplex'
model.add([c1, c2, c3])

status = model.optimize()

print("status:", model.status)
print("objective value:", model.objective.value)
print("----------")
for var_name, var in model.variables.iteritems():
    print(var_name, "=", var.primal)

In [ ]:
import linopy
import numpy as np

# 定义线性规划问题的系数
A = np.array([[1, 2], [-1, 1], [2, 1]])  # 不等式约束的系数矩阵
b = np.array([1, 0, 3])  # 不等式约束的右侧常数
c = np.array([-1, -2])  # 目标函数的系数

# 创建 linopy 模型
model = linopy.Model()

# 添加变量
x = model.add_variables(2)

# 添加约束
for i in range(len(A)):
    model.add_constraints(A[i, :] * x <= b[i])

# 设置目标函数
model.set_objective(c @ x)

# 求解模型
solution = model.optimize()

# 打印结果
print("最优解 x:", solution.x)
print("最优值:", solution.objective_value)


In [ ]:
import pulp as pl
solver_list = pl.listSolvers()
print(solver_list)

In [ ]:
import pulp as pl
solver_list = pl.listSolvers(onlyAvailable=True)
print(solver_list)

In [ ]:
r"""
Author: Louis Luangkesorn <lugerpitt@gmail.com> 2019
https://github.com/lluang

Title: Gemstone Optimization problem

Problem taken from Data, Models, and Decisions by Bertsimas and Freund, 4th Edition
DMD 7.2

## 2 stage problem

- **Scenarios:** $s \in S = (1, 2, 3, 4)$
- **Probability scenario occuring:** $p^s$
- **Cost of steel:** $c$
- **Total steel:** $cap_{steel}$
- **Total molding and assembly hours:** $cap_{molding}, cap_{assembly}^s$
- **Wrench and plier earnings by scenario:** $w^s, p^s$
- **Max demand wrenches and pliers:** $UB_w, UB_p$
- **Decision variables**
  - $(W_{t+1}^s, P_{t+1}^s)$
- **Objective**   $Max \sum_s (p^s * (w^s W_{t+1}^s + p^s P_{t+1}^s) - c$
- **Steel Constraint:** $1.5W_{t+1}^1 + P_{t+1}^1 - C \le 0$
- **Molding Constraint:** $W_{t+1}^1 + P_{t+1}^1 \le cap_{molding}$
- **Assembly Constraint:** $0.3 W_{t+1}^1 + 0.5 P_{t+1}^1  \le cap_{molding}^s$
- **Demand Limit W:** $W \le UB_w$
- **Demand Limit P:** $P \le UB_p$
- **Nonnegativity:** $W, P \ge 0$
"""

import pulp

# parameters
products = ["wrenches", "pliers"]
price = [130, 100]
steel = [1.5, 1]
molding = [1, 1]
assembly = [0.3, 0.5]
capsteel = 27
capmolding = 21
LB = [0, 0]
capacity_ub = [15, 16]
steelprice = 58
scenarios = [0, 1, 2, 3]
pscenario = [0.25, 0.25, 0.25, 0.25]
wrenchearnings = [160, 160, 90, 90]
plierearnings = [100, 100, 100, 100]
capassembly = [8, 10, 8, 10]

production = [(j, i) for j in scenarios for i in products]
pricescenario = [[wrenchearnings[j], plierearnings[j]] for j in scenarios]
priceitems = [item for sublist in pricescenario for item in sublist]

# create dictionaries for the parameters
price_dict = dict(zip(production, priceitems))
capacity_dict = dict(zip(products, capacity_ub * 4))
steel_dict = dict(zip(products, steel))
molding_dict = dict(zip(products, molding))
assembly_dict = dict(zip(products, assembly))

# Create variables and parameters as dictionaries
production_vars = pulp.LpVariable.dicts(
    "production", (scenarios, products), lowBound=0, cat="Continuous"
)
steelpurchase = pulp.LpVariable("steelpurchase", lowBound=0, cat="Continuous")

# Create the 'gemstoneprob' variable to specify
gemstoneprob = pulp.LpProblem("The Gemstone Tool Problem", pulp.LpMaximize)

# The objective function is added to 'gemstoneprob' first
gemstoneprob += (
    pulp.lpSum(
        [
            pscenario[j] * (price_dict[(j, i)] * production_vars[j][i])
            for (j, i) in production
        ]
        - steelpurchase * steelprice
    ),
    "Total cost",
)

for j in scenarios:
    gemstoneprob += pulp.lpSum(
        [steel_dict[i] * production_vars[j][i] for i in products]
    ) - steelpurchase <= 0, ("Steel capacity" + str(j))
    gemstoneprob += pulp.lpSum(
        [molding_dict[i] * production_vars[j][i] for i in products]
    ) <= capmolding, ("molding capacity" + str(j))
    gemstoneprob += pulp.lpSum(
        [assembly_dict[i] * production_vars[j][i] for i in products]
    ) <= capassembly[j], ("assembly capacity" + str(j))
    for i in products:
        gemstoneprob += production_vars[j][i] <= capacity_dict[i], (
            "capacity " + str(i) + str(j)
        )

# Print problem
print(gemstoneprob)

# The problem data is written to an .lp file
# gemstoneprob.writeLP("gemstoneprob.lp")
# The problem is solved using PuLP's choice of Solver
gemstoneprob.solve()
# The status of the solution is printed to the screen
print("Status:", pulp.LpStatus[gemstoneprob.status])

# OUTPUT

# Each of the variables is printed with it's resolved optimum value
for v in gemstoneprob.variables():
    print(v.name, "=", v.varValue)
production = [v.varValue for v in gemstoneprob.variables()]

# The optimised objective function value is printed to the console
print("Total price = ", pulp.value(gemstoneprob.objective))

In [1]:
import pulp as pl
solver_list = pl.listSolvers(onlyAvailable=True)
print(solver_list)

['GLPK_CMD', 'GUROBI', 'GUROBI_CMD', 'PULP_CBC_CMD', 'COIN_CMD', 'HiGHS']


In [29]:
import highspy

In [25]:
import pulp as pl
solver_list = pl.listSolvers()
print(solver_list)

['GLPK_CMD', 'PYGLPK', 'CPLEX_CMD', 'CPLEX_PY', 'GUROBI', 'GUROBI_CMD', 'MOSEK', 'XPRESS', 'XPRESS', 'XPRESS_PY', 'PULP_CBC_CMD', 'COIN_CMD', 'COINMP_DLL', 'CHOCO_CMD', 'MIPCL_CMD', 'SCIP_CMD', 'FSCIP_CMD', 'SCIP_PY', 'HiGHS', 'HiGHS_CMD', 'COPT', 'COPT_DLL', 'COPT_CMD']


In [3]:
import pulp
pulp.__version__

'2.8.0'

In [ ]:
import glpk
import numpy as np

# 定义模型
model = glpk.LPX()

# 定义反应
reactions = ['R1', 'R2', 'R3']
v_min = {'R1': 0, 'R2': 0, 'R3': 0}  # 下限速率
v_max = {'R1': 10, 'R2': 10, 'R3': 10}  # 上限速率

# 添加反应变量到模型
v = {r: model.cols.add(name=r, obj=1.0, bounds=(v_min[r], v_max[r])) for r in reactions}

# 定义代谢物
metabolites = ['M1', 'M2', 'M3', 'M4', 'M5', 'M6']
S = {m: model.cols.add(name=m, obj=0.0, bounds=(0, None)) for m in metabolites}

# 定义平衡方程系数
S_ij = {
    ('R1', 'M1'): -1, ('R1', 'M2'): 1,
    ('R2', 'M2'): -1, ('R2', 'M3'): 1,
    ('R3', 'M3'): -1, ('R3', 'M4'): 1, ('R3', 'M5'): 1, ('R3', 'M6'): -1
}

# 添加平衡方程到模型
for r in reactions:
    for m in metabolites:
        if (r, m) in S_ij:
            model.rows.add(name=f"{r}_{m}", senses='E', rhs=0.0)
            model.rows[-1].matrix = [(v[r], S_ij[(r, m)]), (S[m], -1.0)]

# 设置目标函数（最小化总反应速率）
model.obj.maximize = False

# 求解模型
model.simplex()

# 输出结果
print("Optimal Solution:")
for r in reactions:
    print(f"Reaction {r}: {v[r].value}")
for m in metabolites:
    print(f"Metabolite {m}: {S[m].value}")


In [ ]:
import pulp
pulp.pulpTestAll()

In [12]:
import glpk
glpk.LPX()

<glpk.LPX 0-by-0 at 0x7f51e4043340>

In [22]:
import glpk
glpk.__version__

'0.4.7'

In [24]:
import glpkpy

ModuleNotFoundError: No module named 'glpkpy'